In [1]:
# import tensorflow_datasets as tfds

# # 载入数据集
# dataset, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True, split=['train[:80%]', 'train[80%:]', 'test'])
# # 分别得到训练集、验证集、测试集并打印其数量
# train_dataset, validation_dataset, test_dataset = dataset
# num_train_examples = info.splits['train[:80%]'].num_examples
# num_validation_examples = info.splits['train[80%:]'].num_examples
# num_test_examples = info.splits['test'].num_examples
# print('训练集数量:', num_train_examples)
# print('验证集数量:', num_validation_examples)
# print('测试集数量:', num_test_examples)


In [2]:
import tensorflow_datasets as tfds

# 载入数据集
dataset, info = tfds.load('cats_vs_dogs', with_info=True, as_supervised=True)

# 计算数量并得到训练集、验证集、测试集
num_examples = info.splits['train'].num_examples
num_train_examples = int(num_examples * 0.8)
num_validation_examples = int(num_examples * 0.1)
num_test_examples = num_examples - num_train_examples - num_validation_examples

train_dataset = dataset['train'].take(num_train_examples)
validation_dataset = dataset['train'].skip(num_train_examples).take(num_validation_examples)
test_dataset = dataset['train'].skip(num_train_examples + num_validation_examples)

print('训练集数量:', num_train_examples)
print('验证集数量:', num_validation_examples)
print('测试集数量:', num_test_examples)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/4.0.0.incompleteVYVKU1/cats_vs_dogs-train.tfrecord*...:   0%|…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.0. Subsequent calls will reuse this data.
训练集数量: 18609
验证集数量: 2326
测试集数量: 2327


这里使用的是经典的猫狗分类数据集，该数据集包括25000幅带标签的猫狗照片，其中包含训练集、验证集和测试集。

2. 构建输入流水线
接下来，我们需要利用TensorFlow的数据准备API来构建输入流水线。由于我们将使用迁移学习对预训练的模型进行微调，因此我们需要对图像进行相应的预处理，包括大小调整、归一化等操作。同时，我们可以通过数据增强扩展数据集，更好地训练模型，以提高分类准确率。

以下是一些常用的数据增强操作：

In [3]:
import tensorflow as tf

# 定义一些常用的数据增强操作
def augment(image, label):
    image = tf.image.random_flip_left_right(image) # 随机左右翻转
    image = tf.image.random_brightness(image, max_delta=0.1) # 随机亮度调整
    image = tf.image.random_contrast(image, lower=0.1, upper=0.2) # 随机对比度调整
    image = tf.image.resize(image, [224, 224]) # 调整大小
    image = tf.cast(image, tf.float32) / 255.0 # 归一化
    return image, label

# 定义每个数据集的batch_size
BATCH_SIZE = 64

# 应用数据增强和预处理到训练集
train_dataset = train_dataset.map(augment).shuffle(num_train_examples // 4).batch(BATCH_SIZE)
# 应用预处理到验证集
validation_dataset = validation_dataset.map(lambda image, label: (tf.image.resize(tf.cast(image, tf.float32) / 255.0, [224, 224]), label)).batch(BATCH_SIZE)
# 应用预处理到测试集
test_dataset = test_dataset.map(lambda image, label: (tf.image.resize(tf.cast(image, tf.float32) / 255.0, [224, 224]), label)).batch(BATCH_SIZE)


这里我们定义了augment()函数来进行数据增强和预处理，并应用到了训练集。同时，我们还对验证集和测试集进行了预处理。

3. 加载预训练模型
现在，我们可以加载一个预先训练好的模型，如ResNet50。

In [4]:
from tensorflow.keras.applications.resnet50 import ResNet50

# 加载ResNet50模型
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


94765736/94765736 [==============================] - 0s 0us/step


这里我们使用的是已训练好的ResNet50模型，其中include_top=False表示我们将不包括原始ResNet50模型的顶层（全连接层），并且我们将重定义一个新的顶层来适应我们的猫狗分类任务。

4. 定义新顶层
现在，我们可以定义新的顶层，并将其与ResNet50模型合并。这里我们将使用一个全局平均池化层，一个Dropout层和一个Dense层，并使用softmax激活函数将输出转换为类别概率。

In [8]:
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model

# 定义新顶层
pooling_layer = AveragePooling2D(pool_size=(7, 7))(base_model.output)
flatten_layer = Flatten()(pooling_layer)
dropout_layer = Dropout(0.2)(flatten_layer)
predictions_layer = Dense(2, activation='softmax')(dropout_layer)

# 创建新模型
new_model = Model(inputs=base_model.input, outputs=predictions_layer)

# 锁定ResNet50模型的权重
for layer in base_model.layers:
    layer.trainable = False


这里，我们定义了一个新的AveragePooling2D()层，其将ResNet50的输出轮廓图缩小到更小的尺寸，并使用Flatten()层将其转化为向量形式。Dropout()层是用于减轻过度拟合，最后使用一个Dense()层得到类别概率。

并且我们锁定了预训练模型的权重，以避免它们受到我们的训练策略的影响，从而保留其训练好的特征提取能力。

5. 训练模型
现在，我们可以编译新模型，并使用训练集来训练它。这里我们使用Adam优化器，并使用categorical_crossentropy作为损失函数。

In [9]:
epochs = 3
learning_rate = 1e-2
decay = learning_rate / epochs
momentum = 0.8

# 设置训练参数
new_model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate, decay=decay), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 训练新模型
history = new_model.fit(train_dataset, epochs=epochs, validation_data=validation_dataset)


Epoch 1/3
291/291 [==============================] - 143s 396ms/step - loss: 0.7225 - accuracy: 0.5037 - val_loss: 0.6909 - val_accuracy: 0.5800
Epoch 2/3
291/291 [==============================] - 134s 377ms/step - loss: 0.7131 - accuracy: 0.5050 - val_loss: 0.8979 - val_accuracy: 0.4953
Epoch 3/3
291/291 [==============================] - 135s 380ms/step - loss: 0.7015 - accuracy: 0.5069 - val_loss: 0.8540 - val_accuracy: 0.5258


这里，我们使用fit()函数将训练数据喂给新模型，设置epochs、优化器以及其他训练参数。我们还可以使用evaluate()函数评估模型在测试集上的分类准确率：

In [10]:
test_accuracy = new_model.evaluate(test_dataset)[1]
print('测试集准确率: {:.2f}%'.format(test_accuracy * 100))


37/37 [==============================] - 22s 140ms/step - loss: 0.8319 - accuracy: 0.5342
测试集准确率: 53.42%
